In [1]:
import numpy as np
import pandas as pd
import sympy as sym
from matplotlib import pyplot as plt

In [2]:
from sympy.solvers import solve
from sympy.abc import x
from IPython.display import display

def find_extremums(func, a1, a2):
    dfda1 = func.diff(a1)
    dfda2 = func.diff(a2)
    
    A = func.diff(a1, a1)
    B = func.diff(a1, a2)
    C = func.diff(a2, a2)
    AC_B2 = sym.simplify(A*C-B**2)
    
    points = solve([dfda1, dfda2], [a1, a2], dict=True)
    for point in points:
        print()
        print("solution:")
        display(point)
        print("A*C-B**2:")
        display(AC_B2.subs([(a1, point[a1]),(a2, point[a2])]))
    
    
x, y, a, b, c = sym.symbols("x y a b c")
find_extremums(a*x**3 + b*x + c + a*y**3 + b*y + c, x, y)


solution:


{x: -sqrt(3)*sqrt(-b/a)/3, y: -sqrt(3)*sqrt(-b/a)/3}

A*C-B**2:


12*a**2*(-b/a)


solution:


...

-12*a**2*(-b/a)


solution:


{x: sqrt(3)*sqrt(-b/a)/3, y: sqrt(3)*sqrt(-b/a)/3}

A*C-B**2:


12*a**2*(-b/a)

In [45]:
from sympy import symbols
from sympy.solvers import solve

M1, M2, H = symbols("M1 M2 H", complex=True)
tet1, tet2, phi = symbols("theta1 theta2 phi", complex=False)
Ku1, Ku2, J = symbols("K_u1 K_u2 J", complex=False)
t1, t2 = symbols("t1 t2", complex=False)

Ev1 = \
    -H*M1*sym.cos(phi - tet1) + \
    2*sym.pi*(M1**2)*(sym.cos(tet1)**2) + \
    Ku1*(sym.sin(tet1)**2)

Ev2 = \
    -H*M2*sym.cos(tet2 - phi) + \
    2*sym.pi*(M2**2)*(sym.cos(tet2)**2) + \
    Ku2*(sym.sin(tet2)**2)

Es = J*sym.cos(tet2 - tet1) + t1*Ev1 + t2*Ev2

display(sym.trigsimp(Es.diff(tet1)))
display(Es.diff(tet2))

#solve([
#    sym.trigsimp(Es.diff(tet1))
#], [tet1], simplify=False, implicit=True)

-J*sin(theta1 - theta2) + t1*(-H*M1*sin(phi - theta1) + K_u1*sin(2*theta1) - 2*pi*M1**2*sin(2*theta1))

J*sin(theta1 - theta2) + t2*(-H*M2*sin(phi - theta2) + 2*K_u2*sin(theta2)*cos(theta2) - 4*pi*M2**2*sin(theta2)*cos(theta2))

In [49]:
solve(M1**2*M2 + M2, M1, implicit=False)

[-I, I]